In [1]:
%matplotlib widget
import matplotlib.pyplot as plt

In [2]:
# GEE authentication using service account key stored in local directory
import ee
service_account = 'climate-hazard-demo@data-portal-adaptation.iam.gserviceaccount.com'
credentials = ee.ServiceAccountCredentials(service_account, 'data-portal-adaptation.json')
ee.Initialize(credentials)

In [3]:
import geemap
from ipyleaflet import Marker, MarkerCluster, LayersControl, ZoomControl
from ipywidgets import interact, interactive, interact_manual, Layout
import ipywidgets as widgets
import asyncio
import numpy as np

In [4]:
# From https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20Events.html
class Timer:
    def __init__(self, timeout, callback):
        self._timeout = timeout
        self._callback = callback

    async def _job(self):
        await asyncio.sleep(self._timeout)
        self._callback()

    def start(self):
        self._task = asyncio.ensure_future(self._job())

    def cancel(self):
        self._task.cancel()

def debounce(wait):
    """ Decorator that will postpone a function's
        execution until after `wait` seconds
        have elapsed since the last time it was invoked. """
    def decorator(fn):
        timer = None
        def debounced(*args, **kwargs):
            nonlocal timer
            def call_it():
                fn(*args, **kwargs)
            if timer is not None:
                timer.cancel()
            timer = Timer(wait, call_it)
            timer.start()
        return debounced
    return decorator

In [5]:
# Load data from GEE
MAPprobs = ee.ImageCollection("users/tedwongwri/MAP_probs/ehe_MAPprob_prime01")

In [6]:
initial_threshold = 30
currentYear = 2022
futureYear = 2050
prev_threshold = 30
prev_futureYear = 2050
threshold_value = initial_threshold

In [7]:
futureyear_slider = widgets.IntSlider(
    value=futureYear,
    min=2022,
    max=2080,
    step=1,
    description='Future year',
    continuous_update=False
)

threshold_slider = widgets.IntSlider(
    value=initial_threshold,
    min=5,
    max=50,
    step=5,
    description='EHE threshold',
    continuous_update=False
)

prob_display = widgets.HTML()

In [8]:
currentprobs = MAPprobs.filterMetadata('year', 'equals', currentYear).first()
futureprobs = MAPprobs.filterMetadata('year', 'equals', futureYear).first()

In [9]:
Map = geemap.Map(center=[50.85045, 4.34878], zoom=3,layout=Layout(width='600px', height='400px'), zoom_control=False)
Map.add_basemap('HYBRID')

Map.clear_controls()
layer_control = LayersControl(position='topright')
Map.add_control(layer_control)
Map.add_control(ZoomControl(position='topleft'))

marker = Marker(location=[50.85045, 4.34878], name='location marker')
Map.add_layer(marker)
#Map.setControlVisibility(layerControl=True, fullscreenControl=True, latLngPopup=True)

vizParamsC = {
  'bands': ['threshold_' + str(threshold_value)],
  'min': 0,
  'max': 1,
  'palette': ['909000', '000000'],
  'opacity': 0.5
}
vizParamsF = {
  'bands': ['threshold_' + str(threshold_value)],
  'min': 0,
  'max': 1,
  'palette': ['ff0000', '000000'],
  'opacity': 0.5
}
Map.addLayer(currentprobs, vizParamsC, str(currentYear) + ' Prob(EHE ≥ ' + str(threshold_value) + ')')
Map.addLayer(futureprobs, vizParamsF, str(futureYear) + ' Prob(EHE ≥ ' + str(threshold_value) + ')')


In [10]:
def renameBands(img):
  return img.rename(['05', '10', '15', '20', '25', '30', '35', '40', '45', '50'] )

def prob(threshold, year):
# Returns prob of given EHE threshold for given year, as float betw 0 and 1 inclusive
# Threshold: 5-50 by fives; Year: 2022-2080 by ones    
    data = renameBands(MAPprobs.filterMetadata('year', 'equals', year).first()).reduceRegion(reducer=ee.Reducer.first(), geometry=ee.Geometry.Point(marker.location)).getInfo()
    return data[['0',''][threshold > 5] + str(threshold)]

def update_map():
    global futureprobs
    global futureYear
    global prev_futureYear
    global prev_threshold
    new_visparamsC = {
      'bands': ['threshold_' + str(threshold_value)],
      'min': 0,
      'max': 1,
      'palette': ['909000', '000000'],
      'opacity': 0.5
    }
    new_visparamsF = {
      'bands': ['threshold_' + str(threshold_value)],
      'min': 0,
      'max': 1,
      'palette': ['ff0000', '000000'],
      'opacity': 0.5
    }
    futureprobs = MAPprobs.filterMetadata('year', 'equals', futureYear).first()
    Map.remove_ee_layer(name=str(currentYear) + ' Prob(EHE ≥ ' + str(prev_threshold) + ')')
    Map.remove_ee_layer(name=str(prev_futureYear) + ' Prob(EHE ≥ ' + str(prev_threshold) + ')')
    Map.add_ee_layer(currentprobs, new_visparamsC, str(currentYear) + ' Prob(EHE ≥ ' + str(threshold_value) + ')')
    Map.add_ee_layer(futureprobs, new_visparamsF, str(futureYear) + ' Prob(EHE ≥ ' + str(threshold_value) + ')')
    prev_threshold = threshold_value
    prev_futureYear = futureYear

def update_probdisplay():
    current_prob = prob(threshold_value, currentYear)
    future_prob = prob(threshold_value, futureYear)
    probstring_current = '{:.1f}%'.format(current_prob * 100)
    probstring_future = '{:.1f}%'.format(future_prob * 100)
    ri_string_current = '{:.1f} years'.format(1.0/current_prob)
    ri_string_future = '{:.1f} years'.format(1.0/future_prob)
    newHtml = '<table style="font-size: 150%; border: 1px solid black;"><tr><th>Year</th><th>Prob(EHE ≥ ' + str(threshold_value) + ')</th><th>Recurrence interval</th></tr>'
    newHtml += '<tr><td>' + str(currentYear) + '</td><td style="text-align: center">' + probstring_current + '</td><td>' + ri_string_current + '</td></tr>'
    newHtml += '<tr><td>' + str(futureYear) + '</td><td style="text-align: center">' + probstring_future + '</td><td>' + ri_string_future + '</td></tr></table>'
    prob_display.set_trait('value', newHtml)
    
def update_probplot():
    fig.canvas.draw()
    fig.canvas.flush_events()
    global future_plot
    global plt
    dummy = future_plot.pop(0)
    dummy.remove()
    futureprobs = MAPprobs.filterMetadata('year', 'equals', futureYear).first()
    future_data = renameBands(futureprobs).reduceRegion(reducer=ee.Reducer.first(), geometry=ee.Geometry.Point(marker.location)).getInfo()
    future_plot = plt.plot([str(i) + 'd' for i in future_data.keys()], future_data.values(), label=str(futureYear))
    plt.legend()
    #fig.canvas.manager.set_window_title('')

In [11]:
@debounce(0.5)
def threshold_update(e):
    global threshold_value
    threshold_value = threshold_slider.value
    update_probdisplay()
    update_map()
@debounce(0.5)
def futureyear_update(e):
    global futureYear
    futureYear = futureyear_slider.value
    update_probplot()
    update_probdisplay()
    update_map()
@debounce(0.5)
def marker_update(e):
    update_probplot()
    update_probdisplay()

threshold_slider.observe(threshold_update)
futureyear_slider.observe(futureyear_update)
marker.observe(marker_update)

In [12]:
plt.ioff()
fig = plt.figure(figsize=(5,4))

future_data = renameBands(futureprobs).reduceRegion(reducer=ee.Reducer.first(), geometry=ee.Geometry.Point(marker.location)).getInfo()
current_data = renameBands(currentprobs).reduceRegion(reducer=ee.Reducer.first(), geometry=ee.Geometry.Point(marker.location)).getInfo()
current_plot = plt.plot([str(i) + 'd' for i in current_data.keys()], current_data.values(), label=str(currentYear))
future_plot = plt.plot([str(i) + 'd' for i in future_data.keys()], future_data.values(), label=str(futureyear_slider.value))

plt.xlabel('EHE ≥ threshold')
plt.ylabel('probability')
plt.legend()

plt.ion();

In [13]:
update_probplot()
update_probdisplay()

In [14]:
map_box = widgets.Box([Map], layout=Layout(width='600px', height='400px'))
slider_box = widgets.VBox([futureyear_slider, threshold_slider])
items = [map_box, slider_box, fig.canvas, prob_display]
gridbox = widgets.GridBox(items, layout=Layout(grid_template_columns="repeat(2, 700px)"))
gridbox

GridBox(children=(Box(children=(Map(center=[50.85045, 4.34878], controls=(LayersControl(options=['position'], …